# Efficient ESG evaluation by NLP

チェックリストに沿ったESG評価を自然言語処理で効率化するデモです。  
チェックリストとは、ESGに関する質問のリストです。企業のレポートから適正な回答が得られた個数を数えることで評価します。質問は、Eであれば「気候変動リスクを監視する委員会があるか」「気候変動リスクと機会を特定するプロセスがあるか」などです。  
自然言語処理を用い、質問に対する回答となる箇所を自動で抽出できれば評価業務の効率化ができます。

![demo.PNG](./images/demo.PNG)




デモの手順は以下の通りです。

1. Prepare: PDFファイル(統合報告書)からテキストを読み込みます。
2. Preprocess: テキスト解析しやすいよう整形します。
3. Retrieve: 質問に関連する箇所を抽出します。単語ベースとベクトルベースの2つを行います。
4. (Optional) Answer: 関連する箇所から、質問の回答を抽出します。

### Setup for running the notebook

本Notebookを実行するには、必要なライブラリをインストールした環境が必要です。File > New > Terminalからターミナルを起動し、次のコマンドを実行してください。

* `conda env create -f env-nlp.yml`
* `conda activate env-nlp`

メニューバーのKearnel > Change Kearnel...で`env-nlp`が選択できるようになっているはずです。Kearnelは特定のNotebookを動かすための専用環境のイメージです。もし選択できない場合、次のコマンドを実行しJupyterがKernelを認識できるようにしてください。

```
ipython kernel install --user --name=env-nlp
```

`ipython`がインストールされていない場合、次のコマンドでインストールしてください。

* `conda install ipython`

作成したKernelは他のNotebookでも利用できます。本ノートブックをベースに、いろんなテキストを分析してみましょう！

## 1. Prepare

PDFファイルからテキストを読み込みます。デモでは2019年のトヨタの統合報告書を使用しています。 

In [1]:
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import requests


def set_root():
    root = os.path.join(os.path.realpath("."), "../")
    if root not in sys.path:
        sys.path.append(root)
    return root

ROOT_DIR = Path(set_root())
DATA_DIR = ROOT_DIR / "data"

PDFファイルからのテキスト抽出は、事前に作成しておいた`PDFReader`クラスを使用します。内部的には[`pdfminer.six`](https://github.com/pdfminer/pdfminer.six)を使用しています。  
実装に興味がある方は[ソースコード](https://github.com/icoxfog417/esg-evaluation-by-nlp/blob/master/esg_nlp/data/pdf_reader.py)を参照してください。

In [2]:
from esg_nlp.data.pdf_reader import PDFReader
reader = PDFReader()

In [3]:
file_path = DATA_DIR / f"raw/2019_001_annual_en.pdf"
df = reader.read_to_frame(file_path)

PDF読み込み結果を表示します。

* page: ページ番号
* order: ページ内のセクション番号(登場順にカウント)


In [4]:
df.head(5)

,page,order,content
0,0,0,Page 1
1,0,1,Annual Report \nAnnual Report 2019\nFiscal yea...
2,1,0,Page 1
3,1,1,Table of Contents
4,1,2,1 \nTable of Contents\n2 \nMessage from the Pr...


## 2. Preprocess

PDF読み込み結果は様々なノイズを含んでいるので、前処理を行います。前処理として文字をすべて小文字にするなどします。

In [5]:
import re


# 小文字に統一など
preprocessed = reader.preprocess_frame(df)

`Page 1`など、タイトルのみで文を含んでいないセクションを除外します。文の判定は、ピリオドや疑問符の有無で判定します。

In [6]:
# 文を含んでいないセクションを削除
has_sentence = re.compile("(•)?\s?[A-Za-z](\s)?(\.|;)")
preprocessed = preprocessed[preprocessed["content"].apply(lambda s: re.search(has_sentence, s) is not None)]

preprocessed.head(5)

,page,order,content
4,1,3,the annual report 2019 is intended to communic...
5,1,4,about the pdfthis file is an interactive pdf a...
9,1,8,icons found in each section link to related pa...
16,1,15,toyota’s reports and publications* toyota als...
23,2,2,reforming our company to become a “mobility co...


In [7]:
print(f"Rows are decreased from {len(df)} to {len(preprocessed)}")

Rows are decreased from 747 to 189


In [8]:
preprocessed = preprocessed.assign(length=preprocessed["content"].apply(lambda s: len(s)))
preprocessed.head(5)

,page,order,content,length
4,1,3,the annual report 2019 is intended to communic...,348
5,1,4,about the pdfthis file is an interactive pdf a...,208
9,1,8,icons found in each section link to related pa...,155
16,1,15,toyota’s reports and publications* toyota als...,569
23,2,2,reforming our company to become a “mobility co...,367


## 3. Retrieve

チェックリストの質問に関係しているセクションを抽出します。単語ベースで行う手法と、ベクトルベースで行う手法を紹介します。

質問文は[CDP(Carbon Disclosue Project)](https://www.kaggle.com/c/cdp-unlocking-climate-solutions/overview)の質問を使用しました。CDPはKaggleでコンペティションを実施したことがあり、一部のデータを参照することが可能です(本当は購入しないと参照できません💦)。

In [9]:
# CDPのC2.1の質問
question = "Does your organization have a process for identifying, assessing, and responding to climate-related risks and opportunities ?"
question = question.lower()
language = "en"

## 3.1 Word base retrieval

単純に質問文に含まれている単語を含むセクションを抽出します。文を単語に区切るため[spaCy](https://spacy.io/)を利用します。

In [10]:
from spacy.util import get_lang_class


class Parser():

    def __init__(self, lang):
        self.lang = lang
        self.parser = get_lang_class(self.lang)()
    
    def parse(self, text):
        return self.parser(text)

/home/studio-lab-user/.conda/envs/esg-nlp/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


質問から単語を抽出します。この時、ストップワードと呼ばれる一般的すぎる単語は除外します。

In [11]:
parser = Parser(language)
question_words = [str(t).strip() for t in parser.parse(question) if not t.is_stop and not re.match("\'|\.|\?|\/|\,|\-", t.text)]
question_words

['organization',
 'process',
 'identifying',
 'assessing',
 'responding',
 'climate',
 'related',
 'risks',
 'opportunities']

セクションごとにキーワードが含まれる数をカウントします。

In [12]:
def count_keyword_match(parser, keywords, text):
    tokens = parser.parse(text)
    count = 0
    for t in tokens:
        if str(t).lower().strip() in keywords:
            count += 1
    return count


counted = preprocessed.assign(
            keyword_match=preprocessed["content"].apply(
            lambda s: count_keyword_match(parser, question_words, s))
          )

In [13]:
matched = counted[counted["keyword_match"] > 0]
matched.sort_values(by=["keyword_match"], ascending=False).head(5)

,page,order,content,length,keyword_match
424,37,3,organization and structuretoyota has appointed...,1697,9
392,34,4,making over the decades has been made possible...,2022,4
418,36,5,initiatives related to persons with disabiliti...,3156,3
277,25,7,sustainability meetingreceives reports and del...,264,2
136,12,13,"royalty-free licenses to 23,740 patents relate...",302,2


キーワードの一致数が多い、つまり重要なセクションにフォーカスして調査をすることが可能になります。

## 3.2 Vector base retrieval

キーワードだけでなく、文章の意味を考慮したセクションの抽出を行います。文章をベクトルとして表現し、ベクトル間の距離で意味の近さを判定します。 
文章をベクトル化するために、[Googleの検索でも使用されたBERTという手法](https://blog.google/products/search/search-language-understanding-bert/)を使用します。[Hugging Face](https://huggingface.co/)というライブラリで比較的簡単に扱うことができます。

セクションは非常に長いので、文単位に分割します。

In [14]:
sentences = []
for i, row in preprocessed.iterrows():
    c = row["content"]
    for j, s in enumerate(c.replace("•", ".").replace(";", ".").split(".")):
        sentences.append({
            "page": row["page"],
            "section_order": row["order"],
            "sentence_order": j,
            "sentence": s,
            "length": len(s)
        })

sentences = pd.DataFrame(sentences)
sentences.head(5)

,page,section_order,sentence_order,sentence,length
0,1,3,0,the annual report 2019 is intended to communic...,209
1,1,3,1,more detailed information on toyota’s esg-rel...,112
2,1,3,2,(published december 2019),25
3,1,4,0,about the pdfthis file is an interactive pdf a...,103
4,1,4,1,jump to the beginning of each of the report’s ...,104


In [15]:
len(sentences)

1717

BERTのモデルを使用し、文をベクトル表現に変換します。事前に作成しておいた`encode`関数を使用します。

In [16]:
from esg_nlp.model.encoder import encode

In [17]:
model_name = "distilbert-base-uncased"
embeddings = encode(model_name, sentences["sentence"].values.tolist())

Loading pretrained model...


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Prepair the tokenizer...
Set the pipeline.
Inference start.


100%|██████████| 172/172 [01:36<00:00,  1.79it/s]


In [18]:
embeddings.shape

(1717, 768)

1717個の文それぞれに対し、768次元のベクトルが得られました。質問文もベクトルにします。

In [19]:
query = encode(model_name, question)
query = np.reshape(query, (1, -1))

Loading pretrained model...


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Prepair the tokenizer...
Set the pipeline.
Inference start.


In [20]:
query.shape

(1, 768)

質問文のベクトルと、各文のベクトルとの距離を算出します。

In [21]:
from sklearn.metrics.pairwise import cosine_similarity


distance = cosine_similarity(query, embeddings)
measured = sentences.assign(
            distance=distance.flatten())

measured.sort_values("distance", ascending=False, inplace=True)
measured

,page,section_order,sentence_order,sentence,length,distance
920,25,7,0,sustainability meetingreceives reports and del...,263,0.900027
1589,40,4,5,"we are also promoting activities in social, c...",225,0.881857
1384,35,3,6,when advancing initiatives in safety and heal...,155,0.880402
985,27,5,1,building positive relation-ships with all sta...,242,0.872589
1336,34,4,7,"currently, we have identified three areas—fre...",257,0.871311
...,...,...,...,...,...,...
948,26,5,8,,0,0.165867
950,26,5,10,,0,0.165867
956,26,11,3,,0,0.165867
957,26,11,4,,0,0.165867


質問は気候変動に関するリスクや機会を対策するプロセスを問うものです。その観点では、`manage-ment issues`や`proactively advance initiatives`に関する記述が取れているのは意味を組んでいる印象です。ただ、気候変動に関する文書かというと読み取れないところもあるので、特別に重視したい単語(今回は"climate")がある場合は、いったんキーワードであたりをつけてから使用したほうが有効に見えます。

## 4.(Optional) Answer:

質問回答を学習させたモデルを利用して、直接的に回答を抽出する手法もあります。キーワードで絞り込んだデータから回答を抽出してみましょう。

回答箇所の抽出には、自然言語処理の質問回答の手法を使用します。Wikipediaをベースにした質問回答のデータセット([SQuAD](https://rajpurkar.github.io/SQuAD-explorer/)と呼ばれる)で事前に学習したモデルをお持ちいます。本来はESGに関する質問と回答のデータセットで転移学習したほうが良いですが、今回は学習せずに用います。

In [22]:
from esg_nlp.model.question_answer import answer

1つはキーワードにマッチしているセクションを対象にします。

In [23]:
minimum_match = 1
question_section_pair = matched[matched["keyword_match"] >= minimum_match]["content"].apply(lambda s: (question, s)).tolist()
len(question_section_pair)

48

In [24]:
answers = answer("distilbert-base-uncased-distilled-squad", question_section_pair)
pd.DataFrame(answers).head(5)

Loading pretrained model...
Prepair the tokenizer...
Set the pipeline.
Answer start.


  8%|▊         | 4/48 [00:00<00:04, 10.08it/s]/home/studio-lab-user/.conda/envs/esg-nlp/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:705: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/home/studio-lab-user/.conda/envs/esg-nlp/lib/python3.8/site-packages/transformers/pipelines/question_answering.py:295: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  p_mask = np.asarray(
100%|██████████| 48/48 [00:10<00:00,  4.65it/s]


,score,start,end,answer
0,0.092548,66,72,toyota
1,0.134468,123,154,requires an internet connection
2,0.012953,173,251,developing people message from the cfo capital...
3,0.000197,402,459,joint venture related to the town development ...
4,0.000305,992,996,tnga


`score`が低く、回答があまり正確に取れていないようです。ESGに関する質問は一言で回答できるものではないので、そのまま使うのは難しいかもしれません。

デモは以上です。実際人間が行った時の評価と比べてどうか、ぜひ検証してみてください。